## Offline acquisiton data management using - Pycroscopy Ecosystem:
- Converting to Sidpy datasets:
    - using utilities for sidpy datasets like - fitters, fft etc
- Using scifireaders to read them

#### Contributor(s): Utkarsh Pratiush <utkarshp1161@gmail.com> - 9th May 2025
#### edited - 

### 1. Load the data

In [ ]:
from stemOrchestrator.logging_config   import setup_logging
data_folder  = "."
out_path = data_folder
setup_logging(out_path=out_path) 

In [ ]:
from stemOrchestrator.acquisition import TFacquisition, DMacquisition
from stemOrchestrator.simulation import DMtwin
from stemOrchestrator.process import HAADF_tiff_to_png, tiff_to_png, tiff_metadata, tiff_to_numpy
from autoscript_tem_microscope_client import TemMicroscopeClient
import matplotlib.pyplot as plt
import logging
plot = plt
from typing import Dict

In [ ]:
file_path = "../../data/drift_frame2.tiff"
image_array = tiff_to_numpy(file_path)
metadata_dict = tiff_metadata(file_path)

### 2. convert it to sidpy dataset

In [ ]:
import sidpy
import numpy as np
from sidpy.sid import Dimension

In [ ]:
image = image_array
data_sets = {'Channel_000': sidpy.Dataset.from_array(image.T, name="HAADF")}
data_sets['Channel_000'].data_type = 'image'

ds = data_sets['Channel_000']

ds.original_metadata = metadata_dict['FEI_TITAN_parsed']['Metadata']

pixel_size_x_m = float(ds.original_metadata['BinaryResult']['PixelSize']['X']['#text'])
pixel_size_y_m = float(ds.original_metadata['BinaryResult']['PixelSize']['Y']['#text'])
pixel_size_x_um = pixel_size_x_m * 1e6
pixel_size_y_um = pixel_size_y_m * 1e6

ds.set_dimension(
    0,
    Dimension(np.arange(image.shape[0]) * pixel_size_y_um,
              name='y', units='µm', quantity='Length', dimension_type='spatial')
)

ds.set_dimension(
    1,
    Dimension(np.arange(image.shape[1]) * pixel_size_x_um,
              name='x', units='µm', quantity='Length', dimension_type='spatial')
)

# ds.metadata['note'] = 'The shape of overview image and spectrum image are different in 1st 2 dimensions, possibly due to binning.'


In [ ]:
metadata_dict['FEI_TITAN_parsed']['Metadata']

#### 2.a) use functionalities from sidpy datasets

In [ ]:
view = ds.plot()

In [ ]:
ds